In [1]:
import pandas as pd

In [6]:
df = pd.read_csv("./cmd/addr.txt",header=None,sep='\t',dtype=str,names=['type','code','name'])

In [8]:
province = df[df['type']=='province']

In [21]:
def process(row):
    name = row['name']
    if name.endswith('市'):
        return [name, name.rstrip('市')]
    elif name.endswith('省'):
        return [name, name.rstrip('省')]
    elif name == '内蒙古自治区':
        return [name, '内蒙古省', '内蒙古', '内蒙']
    elif name == '广西壮族自治区':
        return [name, '广西自治区', '广西省', '广西']
    elif name == '西藏自治区':
        return [name, '西藏省', '西藏']
    elif name == '新疆维吾尔自治区':
        return [name,'新疆维吾尔族自治区', '新疆自治区', '新疆省', '新疆']
    return [name]

In [26]:
#province['ext'] = province.apply(process, axis=1)

In [27]:
city = df[df['type']=='city']

In [82]:
zu = open("minzu.txt").read().strip().split('\n')
def _zhou(name):
    x = name.rstrip('自治州')
    for i in zu:
        x = x.replace(i,"")
    ret = [name,x+"自治州",x+"州",x+"市",x]
    if name == '克孜勒苏柯尔克孜自治州': ret.append("克州")
    elif name == '巴音郭楞蒙古自治州': ret.append("巴州")
    elif name == '博尔塔拉蒙古自治州': ret.append("博州")
    return ret
def city_process(row):
    name = row['name']
    if name.endswith('市'):
        ret = [name, name.rstrip('市')]
        if name == '呼和浩特市': ret.append('呼市')
        return ret
    elif name.endswith('盟'):
        return [name, name.rstrip('盟')]
    elif name.endswith("自治州"):
        return _zhou(name)
    elif name.endswith("地区"):
        return [name, name.rstrip("地区")]
    return [name]

In [86]:
#city[~city.name.str.match('.*(市|盟|区|区划|县)$')].name.apply(_zhou)
#city.apply(city_process,axis=1)
city[~city.name.str.match('.*(市|盟|自治州|地区)$')]

,type,code,name
31,city,110100000000,市辖区
32,city,120100000000,市辖区
103,city,310100000000,市辖区
198,city,419000000000,省直辖县级行政区划
212,city,429000000000,省直辖县级行政区划
266,city,469000000000,省直辖县级行政区划
267,city,500100000000,市辖区
268,city,500200000000,县
373,city,659000000000,自治区直辖县级行政区划


In [87]:
county = df[df['type']=='county']

In [110]:
def county_process(row):
    name = row['name']
    if name.endswith('区'):
        return [name, name.rstrip('区')]
    elif name.endswith('县'):
        return [name, name.rstrip('县')]
    elif name.endswith('市'):
        return [name, name.rstrip('市')]
    elif name.endswith('现代产业园'):
        return [name, name.rstrip('现代产业园')]
    elif name.endswith('行政委员会'):
        return [name, name.rstrip('行政委员会')]
    elif name.endswith('管委会'):
        return [name, name.rstrip('管委会')]
    elif name.endswith('自治旗'):
        x = name.rstrip('自治旗')
        return [name, x+'旗', x]
    elif name.endswith('旗'):
        return [name, name.rstrip('旗')]
    elif name == '保定白沟新城':
        return [name, '白沟']
    elif name.endswith('群岛'):
        return [name, name.rstrip('群岛')]
    elif name == '中沙群岛的岛礁及其海域':
        return [name, '中沙群岛', '中沙']
    return [name]

In [111]:
#city[~city.name.str.match('.*(市|盟|区|区划|县)$')].name.apply(_zhou)
#city.apply(city_process,axis=1)
county[~county.name.str.match('.*(县|区|市|现代产业园|行政委员会|管委会|自治旗|旗|保定白沟新城|群岛)$')]

,type,code,name
2693,county,460323000000,中沙群岛的岛礁及其海域


In [106]:
county[county.name.str.match('.*白沟.*')]

,type,code,name
524,county,130672000000,保定白沟新城
